## Why First‑Class Functions Matter for Decorators

- **Decorators** are simply functions that take **another function**, wrap it, and return a new function.  
- That entire mechanism only works because Python lets us treat functions as data.  

## Enhancing Functions: Decorators

- A **decorator** is a **callable** that takes another function, adds behaviour before and/or after it runs, and returns a new callable.  
- They solve cross‑cutting concerns such as logging, timing, permission checks, or retries without cluttering core logic.  
- The magic `@decorator_name` syntax is shorthand for passing the target function to the decorator and re‑binding the original name to the returned wrapper.

## Decorator Anatomy (Manual View)

- **Outer decorator function** accepts the **target function** and creates a **wrapper** inside it.  
- The wrapper usually takes `*args, **kwargs` so it can handle any signature.  
- Wrapper executes optional "before" code, calls the original, maybe does "after" code, and returns the original’s result.  
- Returning the wrapper from the decorator completes the transformation.

Using decorators:
- Manually wrapping illustrates what `@` syntax really does behind the scenes.
- This approach is clear but repetitive: `@` eliminates the manual reassignment step.  

In [14]:
import time

# Target function
def simple_task(sleep_duration):
    time.sleep(sleep_duration)
    print("Running a simple task...")

# Decorator function returns the Wrapper function
def timing_decorator(original_function):
    # The Wrapper function wraps the target function then, decorates and executes it
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = original_function(*args, **kwargs) # calls target function
        duration = time.perf_counter() - start
        print(f"{original_function.__name__} took {duration:.3f}s")
        return result

    return wrapper

simple_task = timing_decorator(simple_task)
simple_task(0.3)

Running a simple task...
simple_task took 0.305s


## The `@` Syntax

- Placing `@decorator_name` directly above `def my_func():` triggers `my_func = decorator_name(my_func)` at *definition* time.  
- After that line is executed, `my_func` refers to the wrapper returned by the decorator, so callers automatically get enhanced behaviour.  
- This keeps the decoration visible and close to the function definition, improving readability.  

In [15]:
@timing_decorator
def another_task():
    print("Running another task...")

another_task()

# Behind the scene:
# - another_task = timing_decorator(another_task)
# - another_task refers to the wrapper function returned by the decorator.


Running another task...
another_task took 0.000s


## Configurable Decorators: Decorators with Arguments

- A **basic decorator** adds fixed behavior; sometimes you need to *configure* that behaviour (e.g. how many retries, which log level).  
- You cannot pass options directly to a plain `@decorator`, because that decorator receives only the **target function**.  
- Solution: call a *factory* that takes options and returns a decorator, then apply it with `@factory(option=value)`.  

In [5]:
def timing_decorator(original_function):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = original_function(*args, **kwargs)
        duration = time.perf_counter() - start
        print(f"{original_function.__name__} took {duration:.3f}s")
        return result

    return wrapper

## The Decorator Factory Pattern

- **Factory function** receives configuration arguments and returns the **actual decorator**.  
- The actual **decorator** still takes the **target function** and builds a **wrapper**.  
- The wrapper can access both the factory’s configuration (via a closure) and the call‑time `*args / **kwargs` for the target function.  
- Three nested layers keep concerns separated: configuration ➜ decoration ➜ runtime.

### Applying Decorators with Arguments

- Use `@factory(arg1, arg2…)` above the function definition.  
- At *definition* time Python calls the factory, gets back a decorator, and applies that decorator to the function.  
- Callers of the function automatically get the behaviour configured by the factory.

## Example: Retry Decorator Factory

- A practical DevOps scenario: retry a flaky operation a configurable number of times.  
- The factory takes `max_attempts`; the wrapper loops until success or until attempts are exhausted, re‑raising the last error.  

In [96]:
import random
from functools import wraps

def retry(max_attempts=3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    print(f"Attempt {attempt}/{max_attempts}")
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f" Error: {e}")
                    if attempt == max_attempts:
                        raise # raise Error when all attempts are made
        return wrapper # decorator returns the wrapper function
        
    return decorator # factory returns the decorator function


# retry(4) is a factory function
# - First, retry(5) is called which must return a decorator.
# - The decorator then receives the function to be decorated.
# - Then, the decorator returns the wrapper function.
@retry(4)
def sometimes_fails():
    if random.random() < 0.7:
        raise RuntimeError("Flaky failure")
    return "Success!"

print(f"Result: {sometimes_fails()}")


Attempt 1/4
 Error: Flaky failure
Attempt 2/4
 Error: Flaky failure
Attempt 3/4
Result: Success!


## Hands-on Exercise

In [97]:
from functools import wraps
 
CURRENT_USER = {"username": "prod-agent", "role": "viewer"}


def require_role(required_role):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if CURRENT_USER.get("role") == required_role:
                print("Permission granted.")
                return func(*args, **kwargs)
            else:
                print(f"Permission denied. Requires '{required_role}'.")
                return None
        return wrapper
    return decorator


@require_role(required_role="admin")
def deploy_service(service_name):
    print(f"Deploying service: {service_name}")
    return "SUCCESS"


result = deploy_service("user-database")
print(f"Final result: {result}")

Permission denied. Requires 'admin'.
Final result: None
